<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/04_SetCovering_Packing_Partitioning/01_SetPacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
#installing library to read xlsx files
!pip install pandas openpyxl

In [37]:
from google.colab import files
uploaded = files.upload()

Saving cost.xlsx to cost (1).xlsx
Saving input.xlsx to input (1).xlsx


In [38]:
import pandas as pd

# Read the data from the Excel file, specifying the correct header row
df = pd.read_excel('input.xlsx', header=1)  # This assumes the second row (index 1) is your header

# Drop the 'Sites' column, as requested
df = df.loc[:, df.columns != 'Sites']

# Print the cleaned dataframe
print(df.to_string())


# Read the data from the Excel file, specifying the correct header row
df2 = pd.read_excel('cost.xlsx')  # This assumes the second row (index 1) is your header

# Drop the 'District' column, as requested
df2 = df2.loc[:, df2.columns != 'District']

# Print the cleaned dataframe
print(df2.to_string())

   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20
0  0  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0
1  1  1  0  0  0  1  1  0  0   0   0   0   0   0   0   0   0   0   0   0
2  0  0  1  1  1  0  0  1  0   0   0   0   0   0   0   0   0   0   0   0
3  0  0  0  0  0  0  1  1  0   1   1   1   1   0   0   0   0   0   0   0
4  0  0  0  0  0  0  0  0  0   0   1   1   1   0   0   1   1   0   0   0
5  0  0  0  0  0  0  0  0  0   1   0   1   0   0   1   1   0   0   0   0
6  0  0  0  0  0  0  0  0  0   0   0   0   1   0   0   0   1   0   0   0
7  0  0  0  0  0  0  0  0  1   0   0   0   0   1   0   0   0   1   0   0
8  0  0  0  0  0  0  0  0  0   0   0   0   0   1   1   0   0   1   1   0
9  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   1   0   1   1
    Cost
0    5.2
1    4.4
2    7.1
3    9.0
4    6.1
5    5.7
6   10.0
7   12.2
8    7.6
9   20.3
10  30.4
11  30.9
12  12.0
13   9.3
14  15.5
15  25.6
16  11.0
17   5.3
18   7.9
19   9.9


In [39]:
# Create a list 'sites' with numbers from 0 to the number of rows in the dataframe
sites = list(range(0, len(df)))
# Print the list 'sites' to verify
print("sites",sites)
# Create a list 'districts' with numbers from 0 to the number of columns in the dataframe
districts = list(range(0, len(df.columns)))
# Print the list 'districts' to verify
print("districts:",districts)
# Export the dataframe to a CSV file and open it in an editor
#df.to_csv('output.csv')

sites [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
districts: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [40]:
#preparing the cost array and populating
cost=np.zeros(len(districts),dtype=float)
for j in range(len(districts)):
  cost[j]=df2.iloc[j,0]
print("cost",cost)


# Create the 'cover' array with the same shape as the dataframe
cover = np.zeros((len(sites), len(districts)), dtype=int)

# Iterate through the dataframe to fill the 'cover' array
for i in range(len(sites)):
    for j in range(len(districts)):
        cover[i][j] = df.iloc[i, j]  # Get the value from the dataframe and assign it to the cover array

# Print the 'cover' array to verify
print("cover array:")
print(cover)

cost [ 5.2  4.4  7.1  9.   6.1  5.7 10.  12.2  7.6 20.3 30.4 30.9 12.   9.3
 15.5 25.6 11.   5.3  7.9  9.9]
cover array:
[[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1]]


In [41]:
#installing pyomo
!pip install -q pyomo

In [42]:
#installing coin or
!apt-get install -y -qq coinor-cbc

In [43]:
#importing pyomo environment
import pyomo.environ as pyo

In [44]:
#creating a model object
model = pyo.ConcreteModel()

In [45]:
#defining variables
model.x = pyo.Var(sites, within=pyo.Binary)
model.y = pyo.Var(districts, within=pyo.Binary)
model.x.pprint()
model.y.pprint()

x : Size=10, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :     1 : False :  True : Binary
      1 :     0 :  None :     1 : False :  True : Binary
      2 :     0 :  None :     1 : False :  True : Binary
      3 :     0 :  None :     1 : False :  True : Binary
      4 :     0 :  None :     1 : False :  True : Binary
      5 :     0 :  None :     1 : False :  True : Binary
      6 :     0 :  None :     1 : False :  True : Binary
      7 :     0 :  None :     1 : False :  True : Binary
      8 :     0 :  None :     1 : False :  True : Binary
      9 :     0 :  None :     1 : False :  True : Binary
y : Size=20, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :     1 : False :  True : Binary
      1 :     0 :  None :     1 : False :  True : Binary
      2 :     0 :  None :     1 : False :  True : Binary
     

In [46]:
# Remove any existing objective component named "obj"
if hasattr(model, 'obj'):
    model.del_component('obj')

#defining objective function
model.obj = pyo.Objective(
    expr = sum(cost[j]*model.y[j] for j in districts),
    sense = pyo.minimize )
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 5.2*y[0] + 4.4*y[1] + 7.1*y[2] + 9.0*y[3] + 6.1*y[4] + 5.7*y[5] + 10.0*y[6] + 12.2*y[7] + 7.6*y[8] + 20.3*y[9] + 30.4*y[10] + 30.9*y[11] + 12.0*y[12] + 9.3*y[13] + 15.5*y[14] + 25.6*y[15] + 11.0*y[16] + 5.3*y[17] + 7.9*y[18] + 9.9*y[19]


In [47]:
#modeling the coverage constraints
model.constraint_coverage = pyo.ConstraintList()
for j in districts:
  model.constraint_coverage.add(sum(model.x[i] for i in sites if cover[i][j]>0)+model.y[j] >=1)
model.constraint_coverage.pprint()

constraint_coverage : Size=20, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, Active=True
    Key : Lower : Body                       : Upper : Active
      1 :   1.0 :                x[1] + y[0] :  +Inf :   True
      2 :   1.0 :         x[0] + x[1] + y[1] :  +Inf :   True
      3 :   1.0 :         x[0] + x[2] + y[2] :  +Inf :   True
      4 :   1.0 :                x[2] + y[3] :  +Inf :   True
      5 :   1.0 :                x[2] + y[4] :  +Inf :   True
      6 :   1.0 :                x[1] + y[5] :  +Inf :   True
      7 :   1.0 :         x[1] + x[3] + y[6] :  +Inf :   True
      8 :   1.0 :         x[2] + x[3] + y[7] :  +Inf :   True
      9 :   1.0 :                x[7] + y[8] :  +Inf :   True
     10 :   1.0 :         x[3] + x[5] + y[9] :  +Inf :   True
     11 :   1.0 :        x[3] + x[4] + y[10] :  +Inf :   True
     12 :   1.0 : x[3] + x[4] + x[5] + y[11] :  +Inf :   True
     13 :   1.0 : x[3] + x[4] + x[6] + y[12] :  +Inf :   True
     14 : 

In [48]:
model.constraint_packing = pyo.ConstraintList()
model.constraint_packing.add(sum(model.x[i] for i in sites)<=4)
model.constraint_packing.pprint()

constraint_packing : Size=1, Index={1}, Active=True
    Key : Lower : Body                                                                : Upper : Active
      1 :  -Inf : x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] :   4.0 :   True


In [49]:
#connecting cbc solver and printing the model
opt = pyo.SolverFactory('cbc')
model.pprint()

2 Var Declarations
    x : Size=10, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary
          9 :     0 :  None :     1 : False :  True : Binary
    y : Size=20, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : Fa

In [50]:
#solve the model
opt_solution = opt.solve(model)

In [51]:
# Print the values of the decision variables
#we are shifting the indices for presentation
print("\nEMS sites opening:")
for i in sites:
    if model.x[i].value > 0:
      print(f"{i+1}: {model.x[i].value}")

print("\nNot covered districts:")
for j in districts:
    if model.y[j].value > 0:
      print(f"{j+1}: {model.y[j].value}")

print(f"Total cost of uncovered districts: {model.obj()}")


EMS sites opening:
3: 1.0
4: 1.0
5: 1.0
9: 1.0

Not covered districts:
1: 1.0
2: 1.0
6: 1.0
9: 1.0
20: 1.0
Total cost of uncovered districts: 32.8
